# create input images for ML

From a geotiff datasource, the 10cm raster data from switzerland, currently "only" covering mots parts of eastern switzerland.

Steps:

1. Create Input Files for cutting (vrt)
1. cut geodata along paths
1. validate images if in correct folder (containing an zebracrossing or not)
1. convert images to png for each band
1. Done: Use the images in ML


See Dockerfile for installed dependencies

Requirements:
* gdal (binaries suffice)

Conda:
* fastai (just for compatability test)
* pyarrow (supporting the feather format)

Pip:
* rasterio
* turfpy
* fiona
* shapely
* pyrosm


## Constants

Run wherever needed, every section should be able to run on its own (well, at least that ist the goal).

In [ ]:
# import constants and helpers
%run Constants.ipynb

# see Constants.ipynb to change start values/settings

# extract crossings locations from osm

within the provided dataset

## load pbf

and restrict to area of interest

In [ ]:
import pyrosm

fetch_pbf()

data_bounds = get_bbox_polygon_for_image(INPUT_DATA_VRT, out_crs=CRS_4326)
osm = pyrosm.OSM(str(OSM_PBF_DEST.absolute()), bounding_box=data_bounds)

Create Area of interest bounding box

## filter nodes to get the locations

In [ ]:
# create the network from osm data
tag_filter = {"highway": ["crossing",]}
CONFIG = dict(
    custom_filter=tag_filter,
    osm_keys_to_keep=["highway", "crossing", "crossing_ref"],
    keep_nodes=True,
    keep_ways=True,
    keep_relations=False,
)
gdf_nodes = osm.get_data_by_custom_criteria(**CONFIG)

In [ ]:
gdf_nodes.head()

## save crossings to disk
and free memory

In [ ]:
import json
gdf_nodes.to_feather(GEOPANDAS_CROSSINGS_RESULT_FEATHER)
gdf_nodes.to_file(GEOPANDAS_CROSSINGS_RESULT_GEOJSON, driver='GeoJSON')

In [ ]:
import gc

del gdf_nodes
del osm
gc.collect()

# extract non-crossing locations from osm

More or less the same as above, but we use the street network nodes.

In [6]:
# import constants and helpers
%run Constants.ipynb

# see Constants.ipynb to change start values/settings

In [2]:
import pyrosm

fetch_pbf()

data_bounds = get_bbox_polygon_for_image(INPUT_DATA_VRT, out_crs=CRS_4326)
osm = pyrosm.OSM(str(OSM_PBF_DEST.absolute()), bounding_box=data_bounds)

In [3]:
gdf_nodes, gdf_edges = osm.get_network(network_type="driving+service", nodes=True)
gdf_nodes.head()

,lon,lat,tags,timestamp,version,changeset,id,geometry
0,8.495231,47.399632,None,1475399106,12,42577489,249091984,POINT (8.49523 47.39963)
1,8.495218,47.399589,"{'crossing': 'traffic_signals', 'crossing_ref'...",1527602166,3,59369294,1264083863,POINT (8.49522 47.39959)
2,8.494893,47.398697,"{'crossing': 'no', 'highway': 'traffic_signals...",1531092005,5,60520366,1247641714,POINT (8.49489 47.39870)
3,8.494834,47.398527,None,1529762079,18,60101624,92206459,POINT (8.49483 47.39853)
4,8.494775,47.398357,"{'crossing': 'traffic_signals', 'crossing_ref'...",1535973608,5,62244007,1770708062,POINT (8.49478 47.39836)


In [4]:
# remove pyrosm instance
import gc
del osm
gc.collect()

63812676

## get all crossings and exclude points within a distance of 25m

In [9]:
# filter for crossing
def crosswalk_tag_filter(tags: dict):
    is_crossing = False
    if not tags:
        return is_crossing
    # see https://wiki.openstreetmap.org/wiki/Key:crossing
    # is_marked_crossing = tags.get("highway") == "crossing" and tags.get("crossing") == "marked"
    # is_zebra_1 = tags.get("crossing") == "zebra"
    # is_zebra_2 = "crossing" in tags and tags.get("crossing_ref") == "zebra"
    if highway_tag := tags.get("highway"):
        is_crossing = highway_tag == "crossing"
    return is_crossing

# crosswalk_tag_filter = lambda x: crosswalk_tag_filter(x)
filter_condition = gdf_nodes.tags.apply(crosswalk_tag_filter)
crossing_locations = gdf_nodes[filter_condition]

In [10]:
avoid_locations = crossing_locations.set_crs(CRS_4326)
avoid_locations = avoid_locations.to_crs(CRS_3857)
avoid_buffered = avoid_locations.buffer(AVOIDING_CROSSINGS_BUFFER_IN_METERS)
avoid_buffered = avoid_buffered.to_crs(CRS_4326)
crossing_locations = crossing_locations.assign(crossing_areas=avoid_buffered)
crossing_locations = crossing_locations.set_geometry('crossing_areas')

In [11]:
# this takes a "while" ;-)

no_crossing_nodes = gdf_nodes.overlay(crossing_locations, how='symmetric_difference')

/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:2196: UserWarning: `keep_geom_type=True` in overlay resulted in 27015 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return geopandas.overlay(


In [12]:
# sanity check
print(len(gdf_nodes))
print(len(crossing_locations))
print(len(no_crossing_nodes))

1484468
27015
1200779


In [13]:
import json
no_crossing_nodes.to_feather(GEOPANDAS_NO_CROSSINGS_RESULT)
no_crossing_nodes.to_file(GEOPANDAS_NO_CROSSINGS_RESULT_GEOJSON, driver='GeoJSON')

/tmp/ipykernel_2445/3660710726.py:2: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  no_crossing_nodes.to_feather(GEOPANDAS_NO_CROSSINGS_RESULT)


In [14]:
import gc

del gdf_nodes
del no_crossing_nodes
gc.collect()

6

# create combined dataframe

for a single source of truth and for better reuse

## combine and reduce datasets

In [33]:
# import constants and helpers
%run Constants.ipynb

# see Constants.ipynb to change start values/settings

In [29]:
import geopandas as gpd
columns = ['geometry']
gdf_crossings = gpd.read_feather(GEOPANDAS_CROSSINGS_RESULT_FEATHER, columns=columns)
gdf_no_crossing = gpd.read_feather(GEOPANDAS_NO_CROSSINGS_RESULT_FEATHER, columns=columns)
gdf_crossings = gdf_crossings.assign(is_crossing=1)
gdf_no_crossing = gdf_no_crossing.assign(is_crossing=0)
crossings_count = len(gdf_crossings)
crossings_absent = len(gdf_no_crossing)
gdf = gdf_no_crossing.append(gdf_crossings)

In [32]:
# sanity check
print("gdf_crossings", len(gdf[gdf['is_crossing'] == 1]) == crossings_count)
print("gdf_no_crossing", len(gdf[gdf['is_crossing'] == 0]) == crossings_absent)
print(gdf.head())

gdf_crossings True
gdf_no_crossing True
                   geometry  is_crossing
0  POINT (8.48187 47.37476)            0
1  POINT (8.48176 47.37487)            0
2  POINT (8.48167 47.37496)            0
3  POINT (8.48150 47.37515)            0
4  POINT (8.48120 47.37546)            0


In [34]:
gdf.to_feather(GEOPANDAS_LABELED_POSITIONS_FEATHER)
gdf.to_file(GEOPANDAS_LABELED_POSITIONS_GEOJSON, driver='GeoJSON')

/tmp/ipykernel_2498/3410018899.py:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  gdf.to_feather(GEOPANDAS_LABELED_POSITIONS_FEATHER)


In [35]:
del gdf
del gdf_no_crossing
del gdf_crossings

# create tiles

In [77]:
# import constants and helpers
%run Constants.ipynb

# see Constants.ipynb to change start values/settings

## prepare geometries

Usage see next section

In [49]:
import geopandas as gpd
gdf = gpd.read_feather(GEOPANDAS_LABELED_POSITIONS_FEATHER)

In [75]:
from shapely.geometry import Point, Polygon, shape

def create_named_buffered_geoms(gdf):
    buffered_geoms = []
    names = []

    for index, l in gdf.iterrows():
        center = l.geometry.centroid
        buff = buffered_shape(shape=l.geometry.centroid, radius_in_meters=(IMAGE_BUFFER_RADIUS_IN_METERS))
        buffered_geoms.append(buff)
        name = f"{center.x}_{center.y}_is_crossing_{l.is_crossing}"
        names.append(name)
    return dict(zip(names, buffered_geoms))

In [76]:
# sanity check
create_named_buffered_geoms(gdf.sample(2))

{'8.5204836_47.3006597_is_crossing_0': <shapely.geometry.polygon.Polygon at 0x7fe44b5dbfa0>,
 '9.1019966_47.1463966_is_crossing_0': <shapely.geometry.polygon.Polygon at 0x7fe49c27bb20>}

## Create virtual concatenated TIF Layer

Using an vrt file.

Currently, the files are in `LV95`.

In [78]:
# skip this step, if an vrt already exists
if not INPUT_DATA_VRT.exists():
    !cd $INPUT_TIF_PATH && gdalbuildvrt -a_srs $CRS_4326 $INPUT_DATA_VRT *.tif

## create tifs

### crossings

In [84]:
selection = gdf[gdf['is_crossing'] == 1]
filename_polygons = create_named_buffered_geoms(selection)

In [85]:
# this takes very very long!
out_image, out_transform = cut_image(INPUT_DATA_VRT, filename_polygon_dict=filename_polygons, destination_folder=IMAGES_TIF_FOLDER_CROSSINGS)

out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.


### non-crossings

In [86]:
selection = gdf[gdf['is_crossing'] == 0].sample(len(crossings))
filename_polygons = create_named_buffered_geoms(selection)

In [87]:
# this takes very very long!
out_image, out_transform = cut_image(INPUT_DATA_VRT, filename_polygon_dict=filename_polygons, destination_folder=IMAGES_TIF_FOLDER_OTHER)

out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.
out of bounds
Input shapes do not overlap raster.


# TODO: CONTINUE HERE!

# convert tifs to more suitable formats

In [ ]:
# import constants and helpers
%run Constants.ipynb

In [ ]:
# experimentally determined "best ration"
scale_ratio = float(255/6000)
scale_ratio

In [ ]:
# this takes a while!
converted_folder = DEST_FOLDER / "converted"
!mkdir -p $converted_folder
!rm -f $converted_folder/*.tif

# convert to uint8, RGB, still with 4 channels
for fn in DEST_FOLDER.glob("*.tif"):
    dest = converted_folder / fn.name
    !rio convert $fn $dest --overwrite --dtype uint8 --scale-ratio $scale_ratio --co photometric=rgb

In [ ]:
TEST_IMAGE = DEST_FOLDER/'0000000001.tif'

In [ ]:
from rasterio import plot
import numpy as np


def reduce_depth(image, display_min, display_max):
    image -= display_min
    image = np.floor_divide(image, (display_min - display_max + 1) / 256)
    image = image.astype(np.uint8)
    return image


with rasterio.open(TEST_IMAGE) as dataset:
    image = dataset.read()
    reduced_image = reduce_depth(image, 0, 65536)
    plot.show(reduced_image[0], cmap="gray")
    # plot.show(dataset.read(4), cmap="gray")

In [ ]:
from rasterio.plot import show_hist
with rasterio.open(TEST_IMAGE) as dataset:
    show_hist(dataset, bins=50, lw=0.0, stacked=True, alpha=0.3, histtype='stepfilled', title="Histogram")

In [ ]:
with rasterio.open(TEST_IMAGE) as dataset:
    image = dataset.read([1,2,3,4])
    reduced_image = reduce_depth(image, 0, 5536)
    plot.show(image)

In [ ]:
# uint8_image = Path(TEST_IMAGE).with_suffix('.uint8.tif')
# uint8_image

In [ ]:
# scale_ratio = float(255/65536)
# print(scale_ratio)
# !rio convert $TEST_IMAGE $uint8_image --overwrite --dtype uint8 --scale-ratio $scale_ratio --co photometric=rgb

In [ ]:
# with rasterio.open(uint8_image) as dataset:
#     image = dataset.read([3])
#     plot.show(image, cmap='Reds_r')


# with rasterio.open(uint8_image) as dataset:
#     image = dataset.read([2])
#     plot.show(image, cmap='Greens_r')


# with rasterio.open(uint8_image) as dataset:
#     image = dataset.read([1])
#     plot.show(image, cmap='Blues_r')

    
# with rasterio.open(uint8_image) as dataset:
#     image = dataset.read([4])
#     plot.show(image, cmap='gray')


In [ ]:
# this is used to find a good scale ratio visually. 
# something between 4000 and 9000 looks to be a good fit.
# uncomment to fine tune this further


# scale_ratio = float(255/65536)
# experimentally determined "best ration"
# scale_ratio = float(255/6000)
# print(scale_ratio)

# # clean directories
# converted_folder = DEST_FOLDER / "converted"
# !mkdir -p $converted_folder
# !rm -f $converted_folder/*.tif

# from fastai.vision.all import *

# # convert to uint8, RGB, still with 4 channels
# converted_fn = []
# for index, fn in enumerate(DEST_FOLDER.glob("*.tif")):
#     dest = converted_folder / fn.name
#     converted_fn.append(dest)
#     if index >= 22:
#         break
#     !rio convert $fn $dest --overwrite --dtype uint8 --scale-ratio $scale_ratio --co photometric=rgb



# image_path = converted_folder
# fnames = get_image_files(image_path)
# fnames.sort()
# print(fnames[0])

# def label_func(fn):
#     return "crossing" # if int(Path(fn).stem) > 100 else "other"

# dls = ImageDataLoaders.from_name_func(
#     image_path,
#     fnames=fnames,
#     bs=12,
#     label_func=label_func,
#     item_tfms=Resize(512),
# )
# dls.show_batch(max_n=20)

In [ ]:
# experimentally determined "best ration"
scale_ratio = float(255/6000)
scale_ratio

In [ ]:
# this takes a while!

converted_folder = DEST_FOLDER / "converted"
!mkdir -p $converted_folder
!rm -f $converted_folder/*.tif

# convert to uint8, RGB, still with 4 channels
for fn in DEST_FOLDER.glob("*.tif"):
    dest = converted_folder / fn.name
    !rio convert $fn $dest --overwrite --dtype uint8 --scale-ratio $scale_ratio --co photometric=rgb

In [ ]:
# verify result is acceptable

from fastai.vision.all import *

image_path = converted_folder
fnames = get_image_files(image_path)
print(fnames[0])

def label_func(fn):
    return "crossing" if int(Path(fn).stem) > 100 else "other"

dls = ImageDataLoaders.from_name_func(
    image_path, fnames = fnames, label_func = label_func, item_tfms=Resize(512)
)
dls.show_batch(max_n=20)

In [ ]:
# this takes a while!

stacked_folder = DEST_FOLDER / "stacked"
!mkdir -p $stacked_folder
!rm -f $stacked_folder/*.tif

# stack bands 1,2,3 and discard the 4th
for fn in converted_folder.glob("*.tif"):
    dest = stacked_folder / fn.name
    !rio stack $fn --bidx 1,2,3 $dest

In [ ]:
# verify result is acceptable

from fastai.vision.all import *

image_path = stacked_folder
fnames = get_image_files(image_path)
print(fnames[0])

def label_func(fn):
    return "crossing"

dls = ImageDataLoaders.from_name_func(
    image_path, fnames = fnames, label_func = label_func, item_tfms=Resize(512)
)
dls.show_batch(max_n=20)    

In [ ]:
# create 3 channel pngs
# hopefully we don't need this, otherwise we loose coordinate information :-(

# converted_png_folder = converted_folder / "png"
# !mkdir -p $converted_png_folder

# from PIL import Image
# for image_path in (converted_folder).glob("*.tif"):
#     destination = Path(converted_png_folder / image_path.stem).with_suffix(".png")
#     with Image.open(image_path,'r') as img:
#         rgb_im = img.convert('RGB')
#         rgb_im.save(destination, format="png")

In [ ]:
codes = ["crossing"]

image_path = stacked_folder

mv $image_path/crossing/*.tif $image_path/

In [ ]:
fnames = get_image_files(image_path)

In [ ]:
def label_func(fn):
    return image_path/"labels"/f"{fn.stem}_P{fn.suffix}"
    # return "stacked" # if int(Path(fn).stem) >= 100 else "other"

crossings_datablock = DataBlock(
    blocks=(ImageBlock, MaskBlock(codes)),
    get_items = get_image_files,
    get_y = label_func,
    splitter=RandomSplitter(),
    # batch_tfms=aug_transforms(size=(512,512)),
    item_tfms=Resize(512),
)

dls = crossings_datablock.dataloaders(image_path, path=image_path.parent, bs=8)
dls.show_batch(max_n=6)

# dls = SegmentationDataLoaders.from_label_func(
#     image_path,
#     bs=8,
#     fnames=fnames,
#     label_func=label_func,
#     # codes=codes,
#     item_tfms=Resize(512),
# )

In [ ]:
dls.show_batch(max_n=2)

In [ ]:
# for img in DEST_FOLDER.glob("*.tif"):
#     convert_banded_image(img)

In [ ]:

with rasterio.open(image_path) as src:
    im_arr = src.read()

In [ ]:
from fastai.vision.all import Path, Image, show

image_path = Path(DEST_FOLDER) / '1.tif'
converted_path = image_path.with_suffix(".png")
assert image_path.exists()

file_out = p.parent / f"{p.stem}.band-{band}.png"
convert_banded_images()

In [ ]:
print(len(out_image))
out_image.shape
out_image[0]

plt.imshow(out_image[1])

In [ ]:
image_array = imread(image_path)

image_array.shape

In [ ]:
from tifffile import imread
import numpy as np
image_array = imread(image_path)

def reduce_depth(image, display_min, display_max):
    image -= display_min
    image = np.floor_divide(image, (display_min - display_max + 1) / 256)
    image = image.astype(np.uint8)
    return image


reduced_image = reduce_depth(out_image[0], 0, 65536)

import matplotlib.pyplot as plt

plt.imshow(reduced_image)

In [ ]:
from tifffile import imread
input = imread(image_path)

def reduce_depth(image, display_min, display_max):
    image -= display_min
    image = np.floor_divide(image, (display_min - display_max + 1) / 256)
    image = image.astype(np.uint8)
    return image

v8 = reduce_depth(input, 0, 65536)

im = Image.fromarray(v8)
im = im.convert('CMYK')
im.save(converted_path.absolute())

In [ ]:
from PIL import Image

# Image.open(image_path.absolute(), mode='r', formats=["TIFF"])

import matplotlib.pyplot as plt
img = plt.imread(image_path.absolute())
img

### OLD STUFF, CODE SNIPPETS

In [ ]:
# rasterio.plot.reshape_as_image
# out_image

In [ ]:
import rasterio
with rasterio.open(INPUT_DATA_VRT) as src:
    print(src.profile)

In [ ]:
print('array type: ',type(thumbnail))
print(thumbnail)

plt.imshow(thumbnail)
plt.colorbar()
plt.title('Overview - Band 4 {}'.format(thumbnail.shape))
plt.xlabel('Column #')
plt.ylabel('Row #')